In [44]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [45]:
data = {'age':[22,25,47,53,46,56,55,60,62,61,18,28,27,29,49,55,25,58,19,18,21,26,40,45,50,54,23],'affordabilty':[1,0,1,0,1,1,0,0,1,1,1,1,0,0,1,1,0,1,0,1,1,0,1,1,1,1,1],'bought_insurance':[0,0,1,0,1,1,0,1,1,1,0,0,0,0,1,1,1,1,0,0,0,0,1,1,1,1,0]}

In [46]:
insurance_data = pd.DataFrame(data)

In [47]:
insurance_data.head()

,age,affordabilty,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,53,0,0
4,46,1,1


In [48]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(insurance_data[['age','affordabilty']],insurance_data['bought_insurance'],test_size = 0.2) 

In [49]:
insurance_data.shape

(27, 3)

In [50]:
len(X_train),len(X_test)

(21, 6)

In [51]:
#scalling the test and train data 
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age']/100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age']/100

In [52]:
X_train_scaled

,age,affordabilty
22,0.40,1
14,0.49,1
13,0.29,0
0,0.22,1
8,0.62,1
19,0.18,1
21,0.26,0
23,0.45,1
10,0.18,1
12,0.27,0


In [53]:
X_test_scaled

,age,affordabilty
4,0.46,1
9,0.61,1
26,0.23,1
24,0.50,1
16,0.25,0
17,0.58,1


In [54]:
#building a simple neural network model where 
#kernel_initializer is weight initialization
#bias_initializer is bias weight intialization
model = keras.Sequential([
    keras.layers.Dense(1,input_shape = (2,), activation = 'sigmoid', kernel_initializer = 'ones',bias_initializer = 'zeros')
])
model.compile( optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy']
              )

In [55]:
model.fit(X_test_scaled,y_test,epochs = 5000)

Streaming output truncated to the last 5000 lines.
Epoch 1544/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.3568 - accuracy: 0.8333
Epoch 1545/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.3567 - accuracy: 0.8333
Epoch 1546/5000
1/1 [==============================] - 0s 18ms/step - loss: 0.3567 - accuracy: 0.8333
Epoch 1547/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.3566 - accuracy: 0.8333
Epoch 1548/5000
1/1 [==============================] - 0s 12ms/step - loss: 0.3566 - accuracy: 0.8333
Epoch 1549/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.3565 - accuracy: 0.8333
Epoch 1550/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.3565 - accuracy: 0.8333
Epoch 1551/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.3564 - accuracy: 0.8333
Epoch 1552/5000
1/1 [==============================] - 0s 12ms/step - loss: 0.3564 - accuracy: 0.8333
Epoch 1553/5000
1/1 [============

In [56]:
model.evaluate(X_test_scaled,y_test)

1/1 [==============================] - 0s 191ms/step - loss: 0.2345 - accuracy: 0.8333


[0.2344718724489212, 0.8333333134651184]

In [57]:
weights,bias = model.get_weights()

In [58]:
weights,bias

(array([[ 5.5794168],
        [-3.0544887]], dtype=float32), array([1.9852319], dtype=float32))

In [59]:
def sigmoid(x):
  import math
  return 1/(1+math.exp(-x))
  

In [60]:
def prediction(age,afford):
  weighted_sum = weights[0]*age + weights[1]*afford
  return sigmoid(weighted_sum)

In [61]:
prediction(0.18,1)

0.11403282577175265

In [62]:
#implementing gradiant descent function from scratch
def logloss(y_true,y_predicted):
  epsilon = 1e-15
  y_predicted_new = [min(i,epsilon) for i in y_predicted]
  y_predicted_new  = [max(i,1-epsilon) for i in y_predicted_new]
  y_predicted_new = np.array(y_predicted_new)
  return -np.mean(y_true*np.log(y_predicted_new) + (1-y_true)*np.log(1-y_predicted_new))

In [63]:
def sigmoid_numpy(X):
  return 1/(1+np.exp(-X))

In [64]:
def gradiant_descent(age,afford,y_true,epochs):
  w1 = w2 = 1
  bias = 0
  learning_rate = 0.5
  n = len(age)
  for i in range(epochs):
    weighted_sum = age*w1 + afford*w2 + bias
    y_predicted   = sigmoid_numpy(weighted_sum)
    loss = logloss(y_true,y_predicted)
    w1d = (1/n)*np.dot(np.transpose(age),(y_predicted - y_true))
    w2d = (1/n)*np.dot(np.transpose(afford),(y_predicted - y_true))
    biasd = np.mean(y_predicted - y_true)
    w1 = w1 - learning_rate*w1d
    w2 = w2 - learning_rate*w2d
    bias = bias - learning_rate*biasd
    print(f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
  return w1,w2,bias


In [65]:
gradiant_descent(X_train_scaled['age'],X_train_scaled['affordabilty'],y_train,1000)

Epoch:0, w1:0.9697068494170068, w2:0.9427249332517388, bias:-0.1459068395561869, loss:19.736900567051933
Epoch:1, w1:0.946442784573238, w2:0.8967078081450343, bias:-0.2732782556740771, loss:19.736900567051933
Epoch:2, w1:0.9296612295358159, w2:0.8613023525534655, bias:-0.38363632306156364, loss:19.736900567051933
Epoch:3, w1:0.9186477374538843, w2:0.8354650386120908, bias:-0.4789079946762397, loss:19.736900567051933
Epoch:4, w1:0.9126240474441278, w2:0.8179549844396784, bias:-0.5611567405084628, loss:19.736900567051933
Epoch:5, w1:0.9108269422739865, w2:0.8074970376875119, bias:-0.6323841797016212, loss:19.736900567051933
Epoch:6, w1:0.9125562431933225, w2:0.8028883697298179, bias:-0.6944127524521643, loss:19.736900567051933
Epoch:7, w1:0.9171971063197216, w2:0.803054182165783, bias:-0.7488340206771286, loss:19.736900567051933
Epoch:8, w1:0.924225068870524, w2:0.8070674612611753, bias:-0.797000058523335, loss:19.736900567051933
Epoch:9, w1:0.9332013153024362, w2:0.8141472506430107, bia

(11.302271959379103, 3.1403146890831177, -6.9013855177231545)